# EDA

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

%matplotlib inline

In [2]:
DATA_PATH = "data/2008.csv"

df = pd.read_csv(DATA_PATH)

### Some Cleaning

Let's keep only "valid" flights, i.e. those that weren't cancelled and have known departure and arrival times.

In [3]:
valid_flight_mask = (df.Cancelled == 0) & ~df.ArrTime.isna() & ~df.DepTime.isna()

print(f"{valid_flight_mask.mean():.1%} of flights are valid")
df = df[valid_flight_mask]

97.8% of flights are valid


### Time

Departure and arrival times are measured in `hours * 100` units.

In [4]:
df[['DepTime', 'ArrTime']].describe()

,DepTime,ArrTime
count,6.858079e+06,6.858079e+06
mean,1.333766e+03,1.481258e+03
std,4.780567e+02,5.052251e+02
min,1.000000e+00,1.000000e+00
25%,9.280000e+02,1.107000e+03
50%,1.325000e+03,1.512000e+03
75%,1.728000e+03,1.909000e+03
max,2.400000e+03,2.400000e+03


In [5]:
through_midnight = (df.ArrTime <= df.DepTime).mean()
print(f"In {through_midnight:.1%} of cases ArrTime <= DepTime")

In 3.2% of cases ArrTime <= DepTime


We will stick to the following assumptions:
- `ArrTime <= DepTime` $\Rightarrow$ arrival happens on the next day
- `ArrTime > DepTime` $\Rightarrow$ departure and arrival both happen on the same day

We'll convert all time-related columns to unix timestamps for convenience.

In [6]:
dates = pd.to_datetime(df.rename(columns={'DayofMonth': 'Day'})[['Year', 'Month', 'Day']])
dateTs = dates.astype(int) // 10**9
df['depTs'] = dateTs + (df.DepTime / 100) * 3600
df['arrTs'] = dateTs + (df.ArrTime / 100) * 3600 + 86400 * (df.ArrTime <= df.DepTime).astype(int)

### Geography

In [7]:
origins = set(df.Origin.unique())
destinations = set(df.Dest.unique())
nodes = origins.union(destinations)
print(f"{len(origins)} unique origins\n{len(destinations)} unique destinations\n{len(nodes)} unique points")

303 unique origins
302 unique destinations
303 unique points


In [8]:
df[~df.Origin.isin(destinations)][['DepTime', 'ArrTime', 'Origin', 'Dest']]

,DepTime,ArrTime,Origin,Dest
3701794,1740.0,1814.0,PUB,COS
3702095,1331.0,1449.0,PUB,COS


No one comes back to Pueblo!